<a href="https://colab.research.google.com/github/Roby290597/nlp_exercise/blob/main/bundestags_reden_analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import requests

In [2]:
import sys
from pathlib import Path

extr = next((str(p/"extr") for p in [Path.cwd()] + list(Path.cwd().parents) if (p/"extr").is_dir()), None)
if extr and extr not in sys.path:
    sys.path.insert(0, extr)
print("extr hinzugefügt zu sys.path:", extr)



from redner_extraction import extract_speeches, extract_all_speakers, get_average_sentiment


extr hinzugefügt zu sys.path: c:\Users\brand\OneDrive\Desktop\AI_Selbststudium\ML_little\bundestag_reden\extr


c:\Users\brand\anaconda3\envs\job\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


oliverguhr/german-sentiment-bert loaded.


In [ ]:
#from sentence_transformers import SentenceTransformer


In [3]:
# Füge Parent-Dir von `extr` zu sys.path hinzu, damit z.B. config.yaml gefunden werden kann

print(os.getcwd())
parent_dir = str(Path(extr).parent)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
print("Parent-Dir hinzugefügt zu sys.path:", parent_dir)

print(os.listdir(parent_dir))


import yaml
# Load YAML data
with open(parent_dir + "/config.yaml", "r") as file:
    data = yaml.safe_load(file)  # safe_load prevents arbitrary code execution

# Modify data (optional)
url = data['data']["xml_file"]

c:\Users\brand\OneDrive\Desktop\AI_Selbststudium\ML_little\bundestag_reden\tasks
Parent-Dir hinzugefügt zu sys.path: c:\Users\brand\OneDrive\Desktop\AI_Selbststudium\ML_little\bundestag_reden
['.git', '.gitignore', 'config.yaml', 'extr', 'fine_tune_summarization.py', 'GPT_fine_tune_summarization.py', 'README.md', 'requirements.txt', 'tasks', 'ToDo.txt', 'web_ui']


### Web Scraping: Neuestes Plenarprotokoll des Bundestags

In [4]:
import xml.etree.ElementTree as ET

response = requests.get(url)

# XML in einen Tree parsen
tree = ET.ElementTree(ET.fromstring(response.content))

# Wurzel-Element abrufen
root = tree.getroot()

print(root.keys())

['vertrieb', 'herstellung', 'sitzung-ort', 'herausgeber', 'issn', 'wahlperiode', 'sitzung-nr', 'sitzung-datum', 'sitzung-start-uhrzeit', 'sitzung-ende-uhrzeit', 'sitzung-naechste-datum', 'start-seitennr']


In [5]:
root.attrib # Alle Attribute des Wurzel-Elements anzeigen

{'vertrieb': 'Bundesanzeiger Verlag GmbH, Postfach 1 0 05 34, 50445 Köln, Telefon (02 21) 97 66 83 40, Fax (02 21) 97 66 83 44, www.bundesanzeiger-verlag.de',
 'herstellung': 'H. Heenemann GmbH  Co. KG, Buch- und Offsetdruckerei, Bessemerstraße 83–91, 12103 Berlin, www.heenemann-druck.de',
 'sitzung-ort': 'Berlin',
 'herausgeber': 'Deutscher Bundestag',
 'issn': '0722-7980',
 'wahlperiode': '21',
 'sitzung-nr': '32',
 'sitzung-datum': '10.10.2025',
 'sitzung-start-uhrzeit': '09:00',
 'sitzung-ende-uhrzeit': '15:13',
 'sitzung-naechste-datum': '15.10.2025',
 'start-seitennr': '3443'}

### Extract all speeches

In [6]:
reden = {}
redner = []
# Beispiel-Ausgabe
for speech in extract_speeches(root):
    #print(f"Redner: {speech['name']}\nRede: {speech['text'][:200]}...\n")
    if speech['name'] not in redner:
        redner.append(speech['name'])
        reden[speech['name']] = speech['text']
    else:
        print("Doppelte Rede:", speech['name'])
        reden[speech['name']] += "\n Nächste Rede:" + speech['text']

Doppelte Rede: Mandy Eißing
Doppelte Rede: Peter Bohnhof
Doppelte Rede: Mahmut Özdemir
Doppelte Rede: Klaus Wiener
Doppelte Rede: Mirze Edis


In [7]:
# Extrahiere alle Redner und ihre Fraktionen
redner_party = []
all_speakers = extract_all_speakers(root)
for speaker in all_speakers:
    print(speaker)
    redner_party.append((speaker['vorname'] + " " + speaker['nachname'], speaker['fraktion']))

{'vorname': 'Bärbel', 'nachname': 'Bas', 'fraktion': '', 'rolle_kurz': 'Bundesministerin BMAS'}
{'vorname': 'Hans-Jürgen', 'nachname': 'Goßner', 'fraktion': 'AfD', 'rolle_kurz': ''}
{'vorname': 'Wilfried', 'nachname': 'Oellers', 'fraktion': 'CDU/CSU', 'rolle_kurz': ''}
{'vorname': 'Ricarda', 'nachname': 'Lang', 'fraktion': 'BÜNDNIS\xa090/DIE GRÜNEN', 'rolle_kurz': ''}
{'vorname': 'Pascal', 'nachname': 'Meiser', 'fraktion': 'Die Linke', 'rolle_kurz': ''}
{'vorname': 'Dagmar', 'nachname': 'Schmidt', 'fraktion': 'SPD', 'rolle_kurz': ''}
{'vorname': 'Peter', 'nachname': 'Bohnhof', 'fraktion': 'AfD', 'rolle_kurz': ''}
{'vorname': 'Sandra', 'nachname': 'Carstensen', 'fraktion': 'CDU/CSU', 'rolle_kurz': ''}
{'vorname': 'Armin', 'nachname': 'Grau', 'fraktion': 'BÜNDNIS\xa090/DIE GRÜNEN', 'rolle_kurz': ''}
{'vorname': 'Nora', 'nachname': 'Seitz', 'fraktion': 'CDU/CSU', 'rolle_kurz': ''}
{'vorname': 'Leif-Erik', 'nachname': 'Holm', 'fraktion': 'AfD', 'rolle_kurz': ''}
{'vorname': 'Peter', 'nachn

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# import numpy as np

# # TF-IDF-Vektor für die Rede von Bärbel Bas erzeugen

# # Rede aus bestehendem Dictionary verwenden
# text = reden.get("Bärbel Bas", "")
# if not text:
#     raise ValueError("Keine Rede für 'Bärbel Bas' im Dictionary 'reden' gefunden.")

# # Deutsches Stopword-Set nutzen, TF-IDF berechnen
# vectorizer = TfidfVectorizer(stop_words="english")
# X = vectorizer.fit_transform([text])  # Ergebnis als sparse matrix
# baerbel_vec = X.toarray()[0]          # dichter Vektor

# print("Vektor-Länge:", baerbel_vec.shape[0])

# # Wichtigste Terme anzeigen (Top 20)
# feature_names = vectorizer.get_feature_names_out()
# top_n = 5
# top_idx = np.argsort(baerbel_vec)[::-1][:top_n]
# print("\nTop-Terme (mit TF-IDF-Werten):")
# for i in top_idx:
#     print(f"{feature_names[i]}: {baerbel_vec[i]:.4f}")


Vektor-Länge: 309

Top-Terme (mit TF-IDF-Werten):
und: 0.3627
auch: 0.3174
wir: 0.2947
die: 0.2720
der: 0.2267


## Sentiment Analysis 

In [8]:
get_average_sentiment("Beifall")


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
[[{'label': 'negative', 'score': 0.9890553951263428}]]


{'label': 'negative', 'score': 0.9890553951263428, 'chunks_analyzed': 1}

In [9]:
result = get_average_sentiment("Beifall")

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
[[{'label': 'negative', 'score': 0.9890553951263428}]]


In [10]:
print("Durchschnittlicher Sentiment-Wert für 'Beifall':", result)

Durchschnittlicher Sentiment-Wert für 'Beifall': {'label': 'negative', 'score': 0.9890553951263428, 'chunks_analyzed': 1}


In [12]:
for i, redner in enumerate(reden.keys()):
    if i >= 5:
        #break
        result = get_average_sentiment(reden[redner])
        print(f"Redner: {redner} - Sentiment: {result['label']} (Score: {result['score']:.4f}, Chunks: {result['chunks_analyzed']})")

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
[[{'label': 'neutral', 'score': 0.9515988826751709}], [{'label': 'neutral', 'score': 0.9851915836334229}], [{'label': 'neutral', 'score': 0.9700415730476379}], [{'label': 'neutral', 'score': 0.9984058737754822}], [{'label': 'positive', 'score': 0.8976163268089294}], [{'label': 'positive', 'score': 0.8420059680938721}], [{'label': 'neutral', 'score': 0.8973153233528137}], [{'label': 'neutral', 'score': 0.9516069889068604}]]
Redner: Dagmar Schmidt - Sentiment: neutral (Score: 0.9367, Chunks: 8)
SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_

KeyboardInterrupt: 